In [2]:
import sympy as sp
import numpy as np
import math

In [ ]:
# -------- gegeven (tabel) --------
h = 35.0        # m
H = 150.0       # m
D = 8.0         # m
t = 0.1         # m

M = 1.0e6       # kg (1000 ton)
J = 1.6e9       # kg m^2
c = 30e3        # N s/m (niet nodig voor natural freq)

E = 210e9       # Pa (staal)
rho = 7850.0    # kg/m^3 (staal)

rho_f = 1025.0  # kg/m^3 (zeewater)

# -------- afgeleid --------
Di = D - 2*t

A = math.pi/4 * (D**2 - Di**2)          # m^2
I = math.pi/64 * (D**4 - Di**4)         # m^4
EI = E * I                               # N m^2



In [ ]:
# -----------------------------
# Symbols
# -----------------------------
z, w = sp.symbols('z omega', real=True)
E, I, T = sp.symbols('E I T', positive=True, real=True)
rho, A = sp.symbols('rho A', positive=True, real=True)

rho_f, Ca, D = sp.symbols('rho_f Ca D', positive=True, real=True)

h, H = sp.symbols('h H', positive=True, real=True)

EI = E*I

# Effective mass per length (submerged) and (above water)
m1 = rho*A + rho_f*Ca*sp.pi*D**2/4   # submerged
m2 = rho*A                            # above water

# Spatial ODE parameters
beta1_4 = (w**2 * m1) / EI
beta2_4 = (w**2 * m2) / EI

# Define alpha^2 and mu^2 exactly as in your derivation
alpha1_sq = (-T/EI + sp.sqrt((T/EI)**2 + 4*beta1_4))/2
mu1_sq    = ( T/EI + sp.sqrt((T/EI)**2 + 4*beta1_4))/2

alpha2_sq = (-T/EI + sp.sqrt((T/EI)**2 + 4*beta2_4))/2
mu2_sq    = ( T/EI + sp.sqrt((T/EI)**2 + 4*beta2_4))/2

alpha1, mu1 = sp.sqrt(alpha1_sq), sp.sqrt(mu1_sq)
alpha2, mu2 = sp.sqrt(alpha2_sq), sp.sqrt(mu2_sq)

# -----------------------------
# General solutions W1(z), W2(z)
# -----------------------------
A1,B1,C1,D1 = sp.symbols('A1 B1 C1 D1')
A2,B2,C2,D2 = sp.symbols('A2 B2 C2 D2')

W1 = A1*sp.cosh(alpha1*z) + B1*sp.sinh(alpha1*z) + C1*sp.cos(mu1*z) + D1*sp.sin(mu1*z)
W2 = A2*sp.cosh(alpha2*z) + B2*sp.sinh(alpha2*z) + C2*sp.cos(mu2*z) + D2*sp.sin(mu2*z)

# Derivatives you will need for beam BCs
W1_1 = sp.diff(W1, z)
W1_2 = sp.diff(W1, z, 2)
W1_3 = sp.diff(W1, z, 3)

W2_1 = sp.diff(W2, z)
W2_2 = sp.diff(W2, z, 2)
W2_3 = sp.diff(W2, z, 3)

# Beam resultants (Euler-Bernoulli with axial tension T):
# Moment:   M = EI * W''
# Shear:    V = EI * W''' + T * W'
M1 = EI*W1_2
V1 = EI*W1_3 + T*W1_1

M2 = EI*W2_2
V2 = EI*W2_3 + T*W2_1

eqs = []

# Bottom example (choose what applies):
eqs += [
    sp.Eq(W1.subs(z, -h), 0),
    sp.Eq(W1_1.subs(z, -h), 0),
]

# Interface z=0 example:
eqs += [
    sp.Eq(W1.subs(z, 0),   W2.subs(z, 0)),
    sp.Eq(W1_1.subs(z, 0), W2_1.subs(z, 0)),
    sp.Eq(M1.subs(z, 0),   M2.subs(z, 0)),
    sp.Eq(V1.subs(z, 0),   V2.subs(z, 0)),
]

# Top example (choose what applies):
eqs += [
    sp.Eq(M2.subs(z, H), 0),
    sp.Eq(V2.subs(z, H), 0),
]